# Converting experimental data to simulation data model

In [ ]:
import pandas as pd
import numpy as np

## Data cleaning

In [ ]:
pd.options.mode.chained_assignment = None

def remove_space(x):
    return x.replace(" ","")

def cleaning(df:pd.DataFrame):
    """Return a cleaned dataframe"""
    # Cleaning the columns name : deleting the spaces
    df.columns = df.columns.str.replace(" ","")

    # Keeping important features
    features = ["cname","frame","time","cellno","schnitzno","birth","lifespan","length","area","angle","grate","width","x","y"]
    df = df[features]
    
    # Removing the spaces in cname
    df["cname"] = df["cname"].apply(remove_space)

    return df

def cleaning2(df):
    """Remove the problematic data"""
    # Removing the rows with cname = 0 0T 0H
    df = df.loc[df["cname"]!="0"]
    df = df.loc[df["cname"]!="0T" ]
    df = df.loc[df["cname"]!="0H"]

    # Calculating the max frame and removing the corresponding rows
    # max = df["frame"].max()
    # df = df.loc[df["frame"]!=max]

    return df



## Data reading

In [ ]:
# df = pd.read_csv("./Data_Durvernoy/coli/141003_MG1655_LB1-fld1.txt",sep="\t")
df = pd.read_csv("./Data_Durvernoy/pseudomonas/100806_pao1.txt",sep="\t")
file_name = "pseudo1"

scale = 65* 1.e-3 # Scale for E.Coli
# all_files = glob.glob("./Data_Durvernoy/pseudomonas/*.txt")
# scale = 64.5* 1.e-3 # Scale for Pseudomonas

df = cleaning(df)

# Applying the scale
df["x"] = df["x"]*scale
df["y"] = df["y"]*scale
df["length"] = df["length"]*scale
df["width"] = df["width"]*scale

df

## File preparation

In [ ]:
# Opening the final file
# file = open("./Data_Duvernoy_converted/Coli/"+ file_name + ".txt","a")
file = open("./Data_Duvernoy_converted/pseudomonas/"+ file_name + ".txt","a")

# Writing the columns names
file.write("N disks\t")
file.write("Disks radius\t")
file.write("time\t")
file.write("Gen\t")
file.write("ks\t")
file.write("kt_bot\t")
file.write("kt_par\t")
file.write("kc\t")
file.write("color\t")
file.write("current length\t")
file.write("div_length\t")
file.write("X")
file.write("\n")


## Writing the new data

In [ ]:

for i,row in df.iterrows():
    # Writing the number of disks 
    file.write(f"{3}\t")

    # Writing the radius
    radius = row["width"]/2
    file.write(f"{radius}\t")

    # Writing the time
    temp = row["time"]
    file.write(f"{temp:.3f}\t")

    # Writing the name
    temp = row["cname"]
    file.write(f"{len(temp)}\t")

    # Writing the stifnesses
    file.write(f"{0}\t")
    file.write(f"{0}\t")
    file.write(f"{0}\t")
    file.write(f"{0}\t")
    file.write(f"{(0,0,0)}\t")

    # Writing the length
    temp = row["length"]
    file.write(f"{temp}\t")
    file.write(f"{temp}\t")

    # writing the disks positions
    xc = row["x"]
    yc = row["y"]

    angle = row["angle"]
    file.write(f"{xc - (temp/2-radius)*np.cos(angle)} ")
    file.write(f"{yc - (temp/2-radius)*np.sin(angle)} ")

    file.write(f"{xc} ")
    file.write(f"{yc} ")

    file.write(f"{xc + (temp/2-radius)*np.cos(angle)} ")
    file.write(f"{yc + (temp/2-radius)*np.sin(angle)} ")
    file.write("\n")

file.close()